In [ ]:
import React, { useState, useEffect, useCallback, useRef } from 'react';
import {
  Button,
  Text,
  Spinner,
  makeStyles,
  shorthands,
  tokens,
} from "@fluentui/react-components";
import * as FluentIcons from "@fluentui/react-icons";

// ... (previous styles remain the same)

const DynamicProgressBar = ({ currentStep }) => {
  const styles = useStyles();
  console.log("DynamicProgressBar received currentStep:", currentStep); // Add this log
  return (
    <div className={styles.progressContainer}>
      <div className={styles.spinnerContainer}>
        <Spinner size="tiny" />
      </div>
      <div className={styles.stepContent}>{currentStep || "Processing..."}</div>
    </div>
  );
};

const EmailGenerator = () => {
  const styles = useStyles();
  // ... (other state variables remain the same)
  const [currentStep, setCurrentStep] = useState("");

  // ... (other functions remain the same)

  const handleUpdate = useCallback((data) => {
    console.log("handleUpdate received data:", data); // Add this log
    if (data.type === 'step') {
      setCurrentStep(data.content);
      console.log("Updated currentStep:", data.content); // Add this log
    } else if (data.type === 'final') {
      finalResponseRef.current = data.content;
      console.log("Received final response:", data.content);
    }
  }, []);

  const handleAction = useCallback(async (userInput) => {
    console.log("handleAction called with input:", userInput);
    setIsProcessing(true);
    setError(null);
    setStatusMessage(null);
    setCurrentStep(""); // Reset currentStep
    finalResponseRef.current = null;
    
    setChatMessages(prevMessages => [...prevMessages, { role: "user", content: userInput }]);
    
    try {
      // ... (rest of the function remains the same)

      let buffer = '';
      while (true) {
        const { value, done } = await reader.read();
        if (done) break;
        
        buffer += decoder.decode(value, { stream: true });
        const lines = buffer.split('\n');
        buffer = lines.pop();

        for (const line of lines) {
          if (line.startsWith('data: ')) {
            const data = line.slice(5).trim();
            if (data === '[DONE]') {
              console.log("Received [DONE] signal");
              // Finalize processing here if needed
            } else {
              try {
                const parsedData = JSON.parse(data);
                handleUpdate(parsedData);
              } catch (error) {
                console.error('Error parsing SSE data:', error);
                console.log('Problematic data:', data);
              }
            }
          }
        }
      }

      // ... (rest of the function remains the same)
    } catch (e) {
      console.error(`Error in handleAction: ${e.message}`);
      setError('Failed to process your request. Please try again.');
    } finally {
      setIsProcessing(false);
      setCurrentStep(""); // Clear currentStep when processing is done
      console.log("Processing completed, isProcessing set to false");
    }
  }, [userConfig, getEmailContent]);

  // ... (rest of the component remains the same)

  if (isChatActive) {
    return (
      <div className={styles.chatPage}>
        <div className={styles.chatMessages}>
          {chatMessages.map((message, index) => (
            <div
              key={index}
              className={`${styles.chatMessage} ${
                message.role === "user" ? styles.userMessage : styles.assistantMessage
              }`}
            >
              {message.role === "user" ? (
                message.content
              ) : (
                <div dangerouslySetInnerHTML={{ __html: message.content }} />
              )}
            </div>
          ))}
        </div>
        {isProcessing && (
          <>
            <DynamicProgressBar currentStep={currentStep} />
            <div>Debug - Current Step: {currentStep}</div> {/* Add this line for debugging */}
          </>
        )}
        <form onSubmit={handleChatSubmit} className={styles.chatInput}>
          <input
            type="text"
            value={chatInput}
            onChange={(e) => setChatInput(e.target.value)}
            placeholder="Type your message..."
            className={styles.chatInputField}
            disabled={isProcessing}
          />
          <Button type="submit" disabled={isProcessing || !chatInput.trim()}>
            Send
          </Button>
        </form>
      </div>
    );
  }

  // ... (rest of the component remains the same)
};

export default EmailGenerator;